# Quantum Gradient - Parameter Shift
## [QHack 2021](https://challenge.qhack.ai/)
### Matt Wright

In [1]:
import pennylane as qml
import numpy as np

In [2]:
def parameter_shift(weights):
    """Compute the gradient of the variational circuit given by the
    ansatz function using the parameter-shift rule.

    Args:
        weights (array): An array of floating-point numbers with size (2, 3).

    Returns:
        array: The gradient of the variational circuit. The shape should match
        the input weights array.
    """
    dev = qml.device("default.qubit", wires=3)

    @qml.qnode(dev)
    def circuit(weights):
        for i in range(len(weights)):
            qml.RX(weights[i, 0], wires=0)
            qml.RY(weights[i, 1], wires=1)
            qml.RZ(weights[i, 2], wires=2)

            qml.CNOT(wires=[0, 1])
            qml.CNOT(wires=[1, 2])
            qml.CNOT(wires=[2, 0])

        return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))

    gradient = np.zeros_like(weights)

    # My solution begins here #
    s = 1.0  # shift factor
    
    N, M = weights.shape
    for i in range(N):
        for j in range(M):
            unit_shift = np.zeros_like(weights)
            unit_shift[i, j] = s
            
            forward_shift = circuit(weights + unit_shift)
            backward_shift = circuit(weights - unit_shift)
            gradient[i, j] = (forward_shift - backward_shift) / (2 * np.sin(s))
    
    # My solution ends here #
    return gradient

## Testing

In [5]:
from test_result import test_result

In [6]:
tol = 10e-3

print('Test 1:')
input_1 = np.array([[1,   0.5, -0.765], 
                    [0.1, 0,   -0.654]])
pred_1 = parameter_shift(input_1).flatten()
ans_1 = np.array([0, 0, 0, 0, -0.4553474723, 0])
test_result(pred_1, ans_1, tol)

print('\nTest 2:')
input_2 = np.array([[0.94, -0.2,   6.03], 
                    [-2.6, -0.058, 1.2]])
pred_2 = parameter_shift(input_2).flatten()
ans_2 = np.array([0.0299518791, -0.0393131888, 0, -0.0340899969, 0.1662797765, 0])
test_result(pred_2, ans_2, tol)

Test 1:
Success!!

Test 2:
Success!!
